This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
from notebook_helpers import usage_string, mprint

load_dotenv()

True

---

# OpenAIFunctionAgent

The `OpenAIFunctionAgent` is a wrapper around OpenAI `functions`: https://platform.openai.com/docs/guides/gpt/function-calling

It decides which tool to use based on the question; it then uses (i.e. calls) that tool (each tool is a callable) and returns the corresponding result. If no tool is used, it returns `None`.

## Define Tools

We're going to define two tools, `fake_weather_api` and `fake_stock_api`.

The `OpenAIFunctionAgent` object will decide which tool to use (i.e. each tool is a callable) and it will return the corresponding response, or `None` if no tool is used.

In [3]:
from llm_workflow.agents import OpenAIFunctionAgent, tool

@tool(
    name="ask_weather",
    description="Use this function to answer questions about the weather for a particular city.",  # noqa
    inputs={
        'location': {
            'type': 'string',
            'description': "The city and state, e.g. San Francisco, CA",
        },
        'unit': {
                'type': 'string',
                'enum': ['celsius', 'fahrenheit'],
                'description': "The temperature unit to use. The model needs to infer this from the `location`.",  # noqa
        },
    },
    required= ['location', 'unit'],
)
def fake_weather_api(location: str, unit: str) -> str:
    return f"The temperature of {location} is 1000 degrees {unit}."

@tool(
    name="ask_stock_price",
    description="Use this function to answer questions about the the stock price for a particular stock symbol.",  # noqa
    inputs={
        'symbol': {
            'type': 'string',
            'description': "The stock symbol, e.g. 'AAPL'",
        },
    },
    required= ['symbol'],
)
def fake_stock_api(symbol: str) -> str:
        return f"The stock price of {symbol} is $1000."

---

## Define/Run Agent

Now let's define and run our Agent.

In [4]:
agent = OpenAIFunctionAgent(
    model_name='gpt-3.5-turbo',
    tools=[fake_weather_api, fake_stock_api],
)

agent("What is the temperature in Seattle WA.")

'The temperature of Seattle, WA is 1000 degrees celsius.'

In [5]:
print(usage_string(agent))

Cost:              $0.00032
Total Tokens:       204
Prompt Tokens:      179
Response Tokens:    25



---

In [6]:
agent("What is the stock price of Apple?")

'The stock price of AAPL is $1000.'

In [7]:
print(usage_string(agent))

Cost:              $0.00062
Total Tokens:       400
Prompt Tokens:      358
Response Tokens:    42



---

In [8]:

agent("Unknow.") is None

True

## `history()`

In [9]:
agent.history()

[ExchangeRecord(uuid='8301a582-c58b-4cc1-b38b-d57f3765af02', timestamp='2023-10-01 02:22:51.227', metadata={'model_name': 'gpt-3.5-turbo', 'tool_name': 'ask_weather', 'tool_args': {'location': 'Seattle, WA', 'unit': 'celsius'}}, cost=0.0003185, total_tokens=204, prompt='What is the temperature in Seattle WA.', response="tool: 'ask_weather' - {'location': 'Seattle, WA', 'unit': 'celsius'}", input_tokens=179, response_tokensns=25),
 ExchangeRecord(uuid='53732108-6113-46c8-95f7-05b36f95cbaf', timestamp='2023-10-01 02:22:53.146', metadata={'model_name': 'gpt-3.5-turbo', 'tool_name': 'ask_stock_price', 'tool_args': {'symbol': 'AAPL'}}, cost=0.00030250000000000003, total_tokens=196, prompt='What is the stock price of Apple?', response="tool: 'ask_stock_price' - {'symbol': 'AAPL'}", input_tokens=179, response_tokensns=17),
 ExchangeRecord(uuid='e8d7c49e-401d-4eb1-a0a3-714cef2602fd', timestamp='2023-10-01 02:22:53.788', metadata={'model_name': 'gpt-3.5-turbo'}, cost=0.00031099999999999997, tot

We can view the function call and arguments in the `metadata` of the `ExchangeRecord`

In [10]:
agent.history()[0].metadata

{'model_name': 'gpt-3.5-turbo',
 'tool_name': 'ask_weather',
 'tool_args': {'location': 'Seattle, WA', 'unit': 'celsius'}}

---